In [ ]:
import numpy as np
import pandas as pd
import cudf
import os
from matplotlib import pyplot as plt
from cuml.solvers import SGD as cumlSGD
from sklearn.linear_model import SGDRegressor

# Select a particular GPU to run the notebook  
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Helper Functions

In [ ]:

from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start


In [ ]:
import gzip
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz'):
    if os.path.exists(cached):
        print('use mortgage data')

        with gzip.open(cached) as f:
            X = np.load(f)
        # the 4th column is 'adj_remaining_months_to_maturity'
        # used as the label
        X = X[:,[i for i in range(X.shape[1]) if i!=4]]
        y = X[:,4:5]
        rindices = np.random.randint(0,X.shape[0]-1,nrows)
        X = X[rindices,:ncols]
        y = y[rindices]

    else:
        print('use random data')
        X = np.random.rand(nrows,ncols)
        y = np.random.randint(0,10,size=(nrows,1))
    train_rows = int(nrows*0.8)
    df_X_train = pd.DataFrame({'fea%d'%i:X[0:train_rows,i] for i in range(X.shape[1])})
    df_X_test = pd.DataFrame({'fea%d'%i:X[train_rows:,i] for i in range(X.shape[1])})
    df_y_train = pd.DataFrame({'fea%d'%i:y[0:train_rows,i] for i in range(y.shape[1])})
    df_y_test = pd.DataFrame({'fea%d'%i:y[train_rows:,i] for i in range(y.shape[1])})
    return df_X_train, df_X_test, df_y_train, df_y_test


In [ ]:

from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=2e-3,with_sign=True):
    a = to_nparray(a).ravel()
    b = to_nparray(b).ravel()
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    error = mean_squared_error(a,b)
    res = error<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x


# Run tests

In [ ]:
%%time
nrows = 2**20
ncols = 399

X_train, X_test, y_train, y_test = load_data(nrows,ncols)
print('data',X_train.shape)
print('label',y_test.shape)



In [ ]:
#set parameters 
learning_rate = 'adaptive'
datatype = np.float32
penalty = 'elasticnet'
loss = 'squared_loss'

Note :setting n_iter to the number of times sgd should iterate through the entire dataset, 
      setting max_iter will not ensure that sgd will iterate through the entire data that many times and 
      therefore a DeprecationWarning will occur


In [ ]:
%%time
# set n_iter to the number of times sgd should iterate through the entire dataset 
# setting max_iter will not ensure that sgd will iterate through the entire data that many times
sk_sgd = SGDRegressor(learning_rate=learning_rate, eta0=0.07,
                       n_iter=500, tol=0.0, fit_intercept=True,
                       penalty=penalty, loss=loss)
sk_sgd.fit(X_train, y_train)


In [ ]:
%%time
y_sk = sk_sgd.predict(X_test)
error_sk = mean_squared_error(y_test,y_sk)


In [ ]:
%%time
X_cudf = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)
y_cudf = np.array(y_train.as_matrix())
y_cudf = y_cudf[:,0]
y_cudf = cudf.Series(y_cudf)

In [ ]:
%%time
cu_sgd = cumlSGD(learning_rate=learning_rate, eta0=0.07, epochs=500,
                 batch_size=512,
                 tol=0.0, penalty=penalty, loss=loss)
cu_sgd.fit(X_cudf, y_cudf)


In [ ]:
%%time
y_pred = cu_sgd.predict(X_cudf_test)
y_pred = to_nparray(y_pred).ravel()
error_cu = mean_squared_error(y_test,y_pred)

In [ ]:
print("SKL MSE(y):")
print(error_sk)
print("CUML MSE(y):")
print(error_cu)